In [6]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline

In [7]:
TRAIN_DIR = "./data/train"
TEST_DIR = "./data/test"
IM_WIDTH = 198
IM_HEIGHT = 198

In [8]:
def plot_images(images,labels):
    n_cols = min(5,len(images))
    n_rows = len(images) // n_cols
    fig = plt.figure(figsize=(8,8))
    
    for i in range(n_rows * n_cols):
        sp = fig.add_subplot(n_rows,n_cols,i+1)
        plt.axis("off")
        plt.imshow(images[i],cmap=plt.cm.gray)
        sp.set_title(labels[i])
    plt.show()

In [4]:
from keras.layers import Input, Dense, Flatten, Conv2D, MaxPool2D, BatchNormalization, Activation, GlobalAvgPool2D
from keras.models import Model

Using TensorFlow backend.


In [11]:
img_input = Input(shape=(IM_HEIGHT,IM_WIDTH,3))
model = Conv2D(filters=32,kernel_size = 3)(img_input)
model = Activation("relu")(model)
model = BatchNormalization()(model)
model = MaxPool2D()(model)

model = Conv2D(filters=64,kernel_size=3)(model)
model = Activation("relu")(model)
model = BatchNormalization()(model)
model = MaxPool2D()(model)

model = Conv2D(filters=64,kernel_size=3)(model)
model = Activation("relu")(model)
model = BatchNormalization()(model)
model = MaxPool2D()(model)

model = Conv2D(filters=128,kernel_size=3)(model)
model = Activation("relu")(model)
model = BatchNormalization()(model)
model = MaxPool2D()(model)

model= Conv2D(filters=128,kernel_size=3)(model)
model = Activation("relu")(model)
model = BatchNormalization()(model)
model = GlobalAvgPool2D()(model)

model = Dense(128)(model)
model = Activation("relu")(model)
model = BatchNormalization()(model)

model = Dense(1)(model)
model = Activation("sigmoid")(model)

In [12]:
model1 = Model(inputs=img_input,outputs=model)
model1.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 198, 198, 3)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 196, 196, 32)      896       
_________________________________________________________________
activation_4 (Activation)    (None, 196, 196, 32)      0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 196, 196, 32)      128       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 98, 98, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 96, 96, 64)        18496     
_________________________________________________________________
activation_5 (Activation)    (None, 96, 96, 64)        0         
__________

In [13]:
from keras.preprocessing.image import ImageDataGenerator
train_data_gen = ImageDataGenerator(rescale=1./255,
                              rotation_range=20,
                              horizontal_flip=True
                             )
 
test_data_gen = ImageDataGenerator(rescale=1./255)
 
batch_size = 32
train_gen = train_data_gen.flow_from_directory(TRAIN_DIR,
                                               target_size=(IM_WIDTH, IM_HEIGHT),
                                               class_mode="binary",
                                               batch_size=batch_size)
test_gen = test_data_gen.flow_from_directory(TEST_DIR,
                                            target_size=(IM_WIDTH, IM_HEIGHT),
                                            class_mode="binary",
                                            batch_size=batch_size)


Found 9636 images belonging to 2 classes.
Found 4145 images belonging to 2 classes.


In [15]:
history = model1.fit_generator(train_gen,steps_per_epoch=len(train_gen.filenames)//batch_size,epochs=16)

Epoch 1/16
301/301 [==============================] - 1903s 6s/step - loss: 0.6174 - acc: 0.6621
Epoch 2/16
301/301 [==============================] - 1968s 7s/step - loss: 0.5192 - acc: 0.7342
Epoch 3/16
301/301 [==============================] - 1966s 7s/step - loss: 0.4771 - acc: 0.7625
Epoch 4/16
301/301 [==============================] - 1981s 7s/step - loss: 0.4429 - acc: 0.7889
Epoch 5/16
301/301 [==============================] - 1966s 7s/step - loss: 0.4129 - acc: 0.8020
Epoch 6/16
301/301 [==============================] - 1965s 7s/step - loss: 0.3891 - acc: 0.8091
Epoch 7/16
301/301 [==============================] - 1963s 7s/step - loss: 0.3724 - acc: 0.8296
Epoch 8/16
301/301 [==============================] - 1963s 7s/step - loss: 0.3478 - acc: 0.8412
Epoch 9/16
301/301 [==============================] - 1963s 7s/step - loss: 0.3243 - acc: 0.8516
Epoch 10/16
301/301 [==============================] - 1965s 7s/step - loss: 0.3080 - acc: 0.8576
Epoch 11/16
301/301 [========

In [19]:
model1.save('GenderModel.h5')